# Résolution problème Teneq1b

In [11]:
using ForwardDiff, JuMP, Ipopt

## Résolution avec ENLSIP-Julia

In [33]:
include("../src/enlsip_functions_castor2.jl")

enlsip

In [4]:
n = 10
m = 10
nb_eq = 0
nb_constraints = 10

10

In [5]:
# Residuals 

function r(x::AbstractArray)
    R = 40
    sum_x = sum(x)
    rx = [x[1]+x[4]-3;
        2*x[1]+x[2]+x[4]+x[7]+x[8]+x[9]+2*x[10]-R;
        2*x[2]+2*x[5]+x[6]+x[7]-8;
        2*x[3]+x[5]-4*R;
        x[1]*x[5]-0.193*x[2]*x[5];
        x[6]*sqrt(x[2])- 0.002597*sqrt(x[2]*x[4]*sum_x);
        x[7]*sqrt(x[4])- 0.003448*sqrt(x[1]*x[4]*sum_x);
        x[8]*x[4]-0.00001799*x[2]*sum_x;
        x[9]*x[4]-0.0002155*x[1]*sqrt(x[3]*sum_x);
        x[10]*(x[4]^2) - 0.00003846*(x[4]^2) * sum_x]
    return rx
end



resT1b = ResidualsEval(0)

function (resT1b::ResidualsEval)(x::Vector, rx::Vector, J::Matrix)

    # Evaluate the residuals
    if abs(resT1b.ctrl) == 1
        rx[:] = r(x)

    # The jacobian is computed analytically
    elseif resT1b.ctrl == 2
        J[:] = ForwardDiff.jacobian(r,x)
    end
    return
end

In [6]:
# Constraints
function c(x::AbstractArray)
    res = [x[1]-0.0001;
        x[2]-0.0001;
        x[3]-0.0001;
        x[4]-0.0001;
        x[5];
        x[6];
        x[7];
        x[8];
        x[9];
        x[10]]
    return res
end

consT1b = ConstraintsEval(0)

function (consT1b::ConstraintsEval)(x::Vector, cx::Vector, A::Matrix)

    # Evaluate the constraints
    if abs(consT1b.ctrl) == 1
        cx[:] = c(x)
    # The jacobian is computed numerically if ctrl is set to 0 on return
    elseif consT1b.ctrl == 2
        A[:] = ForwardDiff.jacobian(c,x)
    end
    return
end

In [34]:
x0 = [1.;1.;20.;1.;0.;0.;0.;0.;0.;1.] 
# x0 = [2.,5.,40.,1.,0.,0.,0.,0.,0.,5.]
e = eps(Float64)
se = sqrt(e)
enlsipTeneq1b = enlsip(x0,resT1b,consT1b,n,m,nb_eq,nb_constraints,ε_abs = e, ε_rel = se, ε_x = se, ε_c = se)


****************************************
*                                      *
*          ENLSIP-JULIA-0.4.0          *
*                                      *
****************************************

Starting point : [1.0, 1.0, 20.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]

Starting point :
    1.000000e+00   1.000000e+00   2.000000e+01   1.000000e+00   0.000000e+00   0.000000e+00   0.000000e+00   0.000000e+00   0.000000e+00   1.000000e+00 

Number of equality constraints   : 0
Number of inequality constraints : 10
Constraints internal scaling     : false


Iteration steps information

iter     objective    cx_sum   reduction     ||p||   dimA  dimJ2     α     conv. speed   max weight   working set
   0  1.559400e+04  0.00e+00   4.63e+03   6.188e+01   4     6   1.61e-01    0.00e+00     1.00e-01    (5,6,7,8)
   1  1.096404e+04  0.00e+00   1.22e-01   1.734e+03   4     6   5.55e-06    8.39e-01     1.00e-01    (5,6,7,8)
   2  1.096392e+04  1.08e-16   1.10e+04   5.183e+01   5     5   1.00e

ENLSIP(40, [2.9976354073619396, 3.9670307223234453, 80.0, 0.002364592638060403, 0.0, 0.001365971440871114, 0.06457258391223862, 3.5312088194740587, 26.430552886097207, 0.0044997904155551095], 5.048729447680022e-29)

In [37]:
enlsipTeneq1b.sol

10-element Vector{Float64}:
  2.9976354073619396
  3.9670307223234453
 80.0
  0.002364592638060403
  0.0
  0.001365971440871114
  0.06457258391223862
  3.5312088194740587
 26.430552886097207
  0.0044997904155551095

## Résolution avec Ipopt

In [42]:
R = 40
x0 = [1.;1.;20.;1.;0.;0.;0.;0.;0.;1.] 
# x0 = [2.,5.,40.,1.,0.,0.,0.,0.,0.,5.]
x_low = [0.0001,0.0001,0.0001,0.0001,0.,0.,0.,0.,0.,0.]
x_upp = Inf*ones(n)

model = Model(Ipopt.Optimizer)

@variable(model, x_low[i] <= x[i=1:n] <= x_upp[i], start = x0[i])

@NLobjective(model, Min, (x[1]+x[4]-3)^2+
        (2*x[1]+x[2]+x[4]+x[7]+x[8]+x[9]+2*x[10]-R)^2+
        (2*x[2]+2*x[5]+x[6]+x[7]-8)^2+
        (2*x[3]+x[5]-4*R)^2+
        (x[1]*x[5]-0.193*x[2]*x[5])^2+
        (x[6]*sqrt(x[2])- 0.002597*sqrt(x[2]*x[4]*sum(x[i] for i=1:n)))^2+
        (x[7]*sqrt(x[4])- 0.003448*sqrt(x[1]*x[4]*sum(x[i] for i=1:n)))^2+
        (x[8]*x[4]-0.00001799*x[2]*sum(x[i] for i=1:n))^2+
        (x[9]*x[4]-0.0002155*x[1]*sqrt(x[3]*sum(x[i] for i=1:n)))^2+
        (x[10]*(x[4]^2) - 0.00003846*(x[4]^2) * sum(x[i] for i=1:n))^2)

optimize!(model)

value.(model[:x])

This is Ipopt version 3.14.4, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:       55

Total number of variables............................:       10
                     variables with only lower bounds:       10
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  1.5589081e+04 0.00e+00 1.01e+02  -1.0 0.00e+00    -  0.00e+00 0.00e+00  

10-element Vector{Float64}:
  2.9969557214716014
  3.967420428501825
 79.99998953532302
  0.0030442781172954065
  2.0929381776805467e-5
  0.0015266065584082261
  0.06359067820261005
  2.660040028255055
 20.211508103382158
  3.5502425203844874